In [3]:
from typing import Annotated
from dotenv import load_dotenv
load_dotenv()

from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain.schema import HumanMessage, AIMessage


llm = init_chat_model("google_genai:gemini-2.0-flash")

In [4]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [5]:
from langchain_core.tools import tool

@tool
def get_stock_price(symbol: str) -> float:
    """Get the stock price for a given symbol.
    
    Args:
        symbol: The stock symbol (e.g., 'MSFT', 'AAPL')
    """
    stock_prices = {
        "MSFT": 200.3,
        "AAPL": 100.4,
        "AMZN": 150.0,
        "RIL": 87.6
    }
    return stock_prices.get(symbol, 0.0)

    

tools = [get_stock_price]

llm = init_chat_model("google_genai:gemini-2.0-flash")
llm_with_tools  = llm.bind_tools(tools)

In [6]:
from langgraph.prebuilt import ToolNode, tools_condition

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

builder = StateGraph(State)
builder.add_node("chatbot", chatbot)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "chatbot")
builder.add_conditional_edges("chatbot", tools_condition)
builder.add_edge("chatbot", END)  # Include this line as in tutorial


graph = builder.compile()

from IPython.display import Image, display
# display(Image(graph.get_graph().draw_mermaid_png()))
# Use ASCII visualization instead
print(graph.get_graph().draw_ascii())

        +-----------+    
        | __start__ |    
        +-----------+    
              *          
              *          
              *          
        +---------+      
        | chatbot |      
        +---------+      
         .         ..    
       ..            .   
      .               .. 
+-------+               .
| tools |             .. 
+-------+            .   
         *         ..    
          **     ..      
            *   .        
        +---------+      
        | __end__ |      
        +---------+      


In [7]:
state = graph.invoke({"messages": [{"role": "user", "content": "What is the price of AMZN stock right now?"}]})
print(state["messages"][-1].content)


150.0


In [8]:
state = graph.invoke({"messages": [{"role": "user", "content": "What is the price of AAPL stock right now?"}]})
print(state)
print(state["messages"][-1].content)

{'messages': [HumanMessage(content='What is the price of AAPL stock right now?', additional_kwargs={}, response_metadata={}, id='1fc4f3c7-f1d0-434b-b1d5-7506363ccfc4'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_stock_price', 'arguments': '{"symbol": "AAPL"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--cbc758c4-c83f-46b8-b763-07b9765c8658-0', tool_calls=[{'name': 'get_stock_price', 'args': {'symbol': 'AAPL'}, 'id': 'e50cb150-7b52-479e-bf3e-60820011a97c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 53, 'output_tokens': 8, 'total_tokens': 61, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='100.4', name='get_stock_price', id='b573ed1c-c1ea-4c74-b557-bb3e40a49ff5', tool_call_id='e50cb150-7b52-479e-bf3e-60820011a97c')]}
100.4
